In [ ]:

# dataset = pd.read_csv('/bin/zombies.csv')

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

train = pd.read_csv('/bin/zombies.csv')
test = pd.read_csv('/bin/zombies.csv')
train.head()

: 

In [ ]:
train.info()


: 

In [ ]:
test.isnull().sum()


: 

In [ ]:
train['clothing'] = train['clothing'].fillna('no') #최빈값
train['documents'] = train['documents'].fillna('no') #최빈값
test['clothing'] = test['clothing'].fillna('no') #최빈값
test['documents'] = test['documents'].fillna('no') #최빈값

train = train.drop(['zombieid'], axis = 1)
test = test.drop(['zombieid'], axis = 1)


: 

In [ ]:

train['zombie'] = train['zombie'].map({'Human' : 0, 'Zombie' : 1})
test['zombie'] = test['zombie'].map({'Human' : 0, 'Zombie' : 1})

train['sex'] = train['sex'].map({'Male' : 0, 'Female' : 1})
test['sex'] = test['sex'].map({'Male' : 0, 'Female' : 1})


train['rurality'] = train['rurality'].map({'Rural' : 0, 'Urban' : 1,'Suburban':2}) 
test['rurality'] = test['rurality'].map({'Rural' : 0, 'Urban' : 1,'Suburban':2})

train['food'] = train['food'].map({'No food' : 0, 'Food' : 1})
test['food'] = test['food'].map({'No food' : 0, 'Food' : 1})

train['medication'] = train['medication'].map({'No medication' : 0, 'Medication' : 1})
test['medication'] = test['medication'].map({'No medication' : 0, 'Medication' : 1})

train['tools'] = train['tools'].map({'No tools' : 0, 'tools' : 1})
test['tools'] = test['tools'].map({'No tools' : 0, 'tools' : 1})

train['firstaid'] = train['firstaid'].map({'No first aid supplies' : 0, 'First aid supplies' : 1})
test['firstaid'] = test['firstaid'].map({'No first aid supplies' : 0, 'First aid supplies' : 1})

train['sanitation'] = train['sanitation'].map({'No sanitation' : 0, 'Sanitation' : 1})
test['sanitation'] = test['sanitation'].map({'No sanitation' : 0, 'Sanitation' : 1})


#'no' : 0,
train['clothing'] = train['clothing'].map({'no' : 0,'Clothing' : 1})
test['clothing'] = test['clothing'].map({'no' : 0,'Clothing' : 1})

train['documents'] = train['documents'].map({'no' : 0,'Documents' : 1})
test['documents'] = test['documents'].map({'no' : 0,'Documents' : 1})


train_x = train[['sex','food','medication','tools', 'firstaid','sanitation', 'clothing', 'documents']]
train_y = train['zombie']


test_x = test[['sex','food','medication','tools', 'firstaid','sanitation', 'clothing', 'documents']]
test_y = test['zombie']

train.head()

: 

In [ ]:
from sklearn.model_selection import train_test_split

X=train[['sex','food','medication','tools', 'firstaid','sanitation', 'clothing', 'documents']]  #독립 변수 X

y=train['zombie']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=14) 

print('train data 개수: ', train_x.shape)

print('test data 개수: ', test_x.shape)



: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# def get_clf_eval0(y_test, pred=None, pred_proba=None):
#     print('오차행렬 \n', confusion_matrix(y_test, pred))
#     print('정확도 :', accuracy_score(y_test, pred))
#     print('정밀도 : ',precision_score(y_test, pred))
#     print('재현율 :', recall_score(y_test, pred))
#     print('f1 score :', f1_score(y_test, pred))
#     print('roc auc score :', roc_auc_score(y_test, pred_proba))

# def get_model_train_eval(model, ftr_train = None, ftr_test = None, tgt_train=None, tgt_test=None):
#     model.fit(ftr_train, tgt_train)
#     pred = model.predict(ftr_test)
#     pred_proba = model.predict_proba(ftr_test)[:,1]
#     get_clf_eval(tgt_test, pred, pred_proba)

lgbm_wrapper = LGBMClassifier(n_estimators=400)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
evals = [(test_x, test_y)]
lgbm_wrapper.fit(train_x, train_y, early_stopping_rounds=100, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(test_x)
pred_proba = lgbm_wrapper.predict_proba(test_x)[:, 1]


: 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


get_clf_eval(test_y, preds, pred_proba)



: 

: 

In [ ]:
import joblib

model=lgbm_wrapper
joblib.dump(model,'/bin/prediction2.pkl')
load_model=joblib.load('/bin/prediction2.pkl')


: 